In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# For Data Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, StandardScaler

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

from sklearn.metrics import classification_report
from sklearn.metrics import precision_score, recall_score, confusion_matrix, roc_curve, precision_recall_curve, accuracy_score

In [ ]:
data = pd.read_csv('/kaggle/input/breast-cancer-wisconsin-data/data.csv')
df=pd.DataFrame(data)
data.head(10)
col_list=list(df)
print(col_list)

In [ ]:
data.info()

In [ ]:
plt.figure(figsize=(10, 7))
plt.title('Count Plot')
sns.countplot(x=data['diagnosis'], palette='Blues_r')

In [ ]:
le = LabelEncoder()
data.diagnosis = le.fit_transform(data.diagnosis)

In [ ]:
X = data.drop(['id', 'diagnosis', 'Unnamed: 32'], axis=1)
y = data['diagnosis']

In [ ]:
X.describe()

In [ ]:
from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler()
X_1 = standardizer.fit_transform(X)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape)

In [ ]:
for col in data.columns:
    print(col)

# # Feature selection based on chi-square 

In [ ]:
# These are chi-square score values , higher the value higher the importance of the target variable
chi_score=chi2(X,y)
print(chi_score)
chi_values = pd.Series(chi_score[0], index=X.columns)
chi_values.sort_values(ascending=False, inplace=True)
chi_values.plot.bar()

In [ ]:
chi2_features = SelectKBest(chi2, k = 5)

X_train_features = chi2_features.fit_transform(X_train, y_train)
X_test_features = chi2_features.transform(X_test)
print('Number of Original Features:', X_train.shape[1])
print('Selected Number of Features:', X_train_features.shape[1])

In [ ]:
print('Score list:', chi2_features.scores_)
print('Feature list:',X_train.columns)

# Confusion matrix

In [ ]:
def metrics(X,CV_clf):
    y_pred = CV_clf.predict(X)
    cm = confusion_matrix(y_test, y_pred)
    tn = cm[0,0]
    fp = cm[0,1]
    fn = cm[1,0]
    tp = cm[1,1]
    Accuracy=(tp+tn)/(tp+tn+fp+fn)
    Sensitivity=tp/(tp+fn)
    Specificity=tn/(tn+fp)
    Precision=tp/(tp+fp)
    F_measure=2*tp/(2*tp+fp+fn)
    print('Accuracy=%.4f'%Accuracy)
    print('Sensitivity=%.4f'%Sensitivity) # as the same as recall
    print('Specificity=%.4f'%Specificity)
    print('Precision=%.4f'%Precision)
    print('F-measure=%.4f'%F_measure)
    return Accuracy, Sensitivity, Specificity, Precision, F_measure
 #   plot_confusion_matrix(CV_clf, X_test, y_test)

# KNN Classifier

In [ ]:
#Original data model KNN
# Importing KNN Classifier
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=15)
knn_model.fit(X_train, y_train)
knn_model.score(X_test,y_test)

y_pred=knn_model.predict(X_test)
#print(classification_report(y_test, y_pred))


knn_model_metrics = metrics(X_test,knn_model)

# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')
#print("Our accuracy is {}%".format(((cm[0][0] + cm[1][1])/57)*100))
#sum = 0
#for i in range(cm.shape[0]):
#    sum += cm[i][i]  
#accuracy = sum/X_test.shape[0]
#print('Accuracy=' , accuracy)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - KNN
knn_chi_square = KNeighborsClassifier(n_neighbors=15) 
knn_chi_square.fit(X_train_features,y_train)
knn_chi_square.score(X_test_features, y_test)


In [ ]:
# CHisquare KNN model maetrics
y_pred_chi=knn_chi_square.predict(X_test_features)
knn_model_metrics_chi = metrics(X_test_features,knn_chi_square)

# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred_chi)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(knn_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(knn_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(knn_chi_square.score(X_test_features, y_test)*100))

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
log_model = LogisticRegression(random_state=42)
#log_model.fit(X_train, y_train)

# # Support Vector Machine

In [ ]:
#Original data model SVM
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train, y_train)
y_pred=svm_model.predict(X_test)
#print(classification_report(y_test, y_pred))

svm_model_metrics = metrics(X_test,svm_model)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - SVM
svm_chi_square = SVC(kernel='linear', random_state=42)
svm_chi_square.fit(X_train_features,y_train)
svm_chi_square.score(X_test_features, y_test)

In [ ]:
# CHisquare SVM model maetrics
y_pred_chi=svm_chi_square.predict(X_test_features)
svm_model_metrics_chi = metrics(X_test_features,svm_chi_square)

# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred_chi)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(svm_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(svm_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(svm_chi_square.score(X_test_features, y_test)*100))

# #Naive Bays

In [ ]:
#Original data model Naive Bays
# Importing Naive Bayes
from sklearn.naive_bayes import GaussianNB

naive_model = GaussianNB()
naive_model.fit(X_train, y_train)
y_pred=naive_model.predict(X_test)
print(classification_report(y_test, y_pred))

naive_model_metrics = metrics(X_test,naive_model)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - NaiveBays
bays_chi_square = GaussianNB()
bays_chi_square.fit(X_train_features,y_train)
bays_chi_square.score(X_test_features, y_test)

In [ ]:
# CHisquare Naive Bays model maetrics
y_pred_chi=bays_chi_square.predict(X_test_features)
bays_model_metrics_chi = metrics(X_test_features,bays_chi_square)

# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred_chi)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(naive_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(naive_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(bays_chi_square.score(X_test_features, y_test)*100))

# # Decision Tree

In [ ]:
# Importing Decision Tree
from sklearn.tree import DecisionTreeClassifier

decision_model = DecisionTreeClassifier(criterion='entropy', random_state=42)
decision_model.fit(X_train, y_train)
y_pred=decision_model.predict(X_test)
print(classification_report(y_test, y_pred))

decision_model_metrics = metrics(X_test,decision_model)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - Decision tree
decision_chi_square =DecisionTreeClassifier(criterion='entropy', random_state=42)
decision_chi_square.fit(X_train_features,y_train)
decision_chi_square.score(X_test_features, y_test)

In [ ]:
# CHisquare Decision Tree model maetrics
y_pred_chi=decision_chi_square.predict(X_test_features)
decision_model_metrics_chi = metrics(X_test_features,decision_chi_square)

# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred_chi)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(decision_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(decision_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(decision_chi_square.score(X_test_features, y_test)*100))

# # Bagging Decision tree

In [ ]:
# Imprting Bagging Classifier
from sklearn.ensemble import BaggingClassifier
bg_model = BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=20)
bg_model.fit(X_train, y_train)
y_pred=bg_model.predict(X_test)
print(classification_report(y_test, y_pred))

bg_model_metrics = metrics(X_test,bg_model)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - bagging
bg_chi_square =BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5,max_features=1.0,n_estimators=20)
bg_chi_square.fit(X_train_features,y_train)
bg_chi_square.score(X_test_features, y_test)

In [ ]:
# CHisquare Bagging model maetrics
y_pred_chi=bg_chi_square.predict(X_test_features)
bg_model_metrics_chi = metrics(X_test_features,bg_chi_square)

# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred_chi)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(bg_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(bg_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(bg_chi_square.score(X_test_features, y_test)*100))

# # Boosting Decision tree

In [ ]:
# Importing Boosting Algorithms
from sklearn.ensemble import AdaBoostClassifier

adb_model = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),n_estimators=10,learning_rate=0.6)
adb_model.fit(X_train, y_train)
y_pred=adb_model.predict(X_test)
print(classification_report(y_test, y_pred))

adb_model_metrics = metrics(X_test,adb_model)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - boosting
adb_chi_square = AdaBoostClassifier(DecisionTreeClassifier(min_samples_split=10,max_depth=4),n_estimators=10,learning_rate=0.6)
adb_chi_square.fit(X_train_features,y_train)


In [ ]:
# CHisquare Boosting model maetrics
y_pred_chi=adb_chi_square.predict(X_test_features)
adb_model_metrics_chi = metrics(X_test_features,adb_chi_square)

# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred_chi)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(adb_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(adb_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(adb_chi_square.score(X_test_features, y_test)*100))

# # Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=40, max_depth=10)
rf_model.fit(X_train, y_train)
y_pred=rf_model.predict(X_test)
print(classification_report(y_test, y_pred))

rf_model_metrics = metrics(X_test,rf_model)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - Random forest
rf_chi_square = RandomForestClassifier(n_estimators=40, max_depth=10)
rf_chi_square.fit(X_train_features,y_train)


In [ ]:
# CHisquare RandomForest model maetrics
y_pred_chi=rf_chi_square.predict(X_test_features)
rf_model_metrics_chi = metrics(X_test_features,rf_chi_square)

# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred_chi)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(rf_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(rf_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(rf_chi_square.score(X_test_features, y_test)*100))

# # voting classifier

In [ ]:
# Importing Voting Classifier
from sklearn.ensemble import VotingClassifier

evc=VotingClassifier(estimators=[('mnb',naive_model),('adb',adb_model),('rf',rf_model),('svm',svm_model)],voting='hard')
evc.fit(X_train, y_train)
y_pred=evc.predict(X_test)
print(classification_report(y_test, y_pred))

evc_metrics = metrics(X_test,evc)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# chi-square data model
from sklearn.ensemble import VotingClassifier
chi_square_evc=VotingClassifier(estimators=[('dtrre',decision_chi_square),('adb',adb_chi_square),('rf',rf_chi_square),('svm',svm_chi_square)],voting='hard')
chi_square_evc.fit(X_train, y_train)

y_pred=evc.predict(X_test)
print(classification_report(y_test, y_pred))

evc_metrics = metrics(X_test,chi_square_evc)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(chi_square_evc.score(X_test, y_test)*100))
print("Score on Train: "+ str(chi_square_evc.score(X_train, y_train)*100))
#print("Score using ChiSquare method: "+ str(chi_square_evc.score(X_test_features, y_test)*100))

# # another combinationnnnnnnnn

In [ ]:
# chi-square data model
from sklearn.ensemble import VotingClassifier
chi_square_evc1=VotingClassifier(estimators=[('bg',bg_chi_square),('adb',adb_chi_square),('rf',rf_chi_square)],voting='hard')
chi_square_evc1.fit(X_train, y_train)

y_pred=evc.predict(X_test)
print(classification_report(y_test, y_pred))

evc_metrics = metrics(X_test,chi_square_evc1)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
print("Score on Test: " + str(chi_square_evc1.score(X_test, y_test)*100))
print("Score on Train: "+ str(chi_square_evc1.score(X_train, y_train)*100))
#print("Score using ChiSquare method: "+ str(chi_square_evc.score(X_test_features, y_test)*100))

# CatBoost (Decision tree)

In [ ]:
from catboost import CatBoostClassifier

catboost_model= CatBoostClassifier(iterations=10)
catboost_model.fit(X_train, y_train)
y_pred=catboost_model.predict(X_test)
print(classification_report(y_test, y_pred))

catboost_model_metrics = metrics(X_test,catboost_model)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - CATboosting
catboost_chi_square =  CatBoostClassifier(iterations=20)
catboost_chi_square.fit(X_train_features,y_train)


In [ ]:
print("Score on Test: " + str(catboost_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(catboost_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(catboost_chi_square.score(X_test_features, y_test)*100))

# LightGBM classifier (better than XGBoost)(faster)

In [ ]:
import lightgbm as lgb
lgbm_model= lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
lgbm_model.fit(X_train, y_train)
y_pred=lgbm_model.predict(X_test)
print(classification_report(y_test, y_pred))


lgbm_model_metrics = metrics(X_test,lgbm_model)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
# Chi - Square data Model  - LGBM
lgbm_chi_square =  lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
lgbm_chi_square.fit(X_train_features,y_train)

In [ ]:
print("Score on Test: " + str(lgbm_model.score(X_test, y_test)*100))
print("Score on Train: "+ str(lgbm_model.score(X_train, y_train)*100))
print("Score using ChiSquare method: "+ str(lgbm_chi_square.score(X_test_features, y_test)*100))

# Ensemble  voting (top 4)

In [ ]:
# Importing Voting Classifier
from sklearn.ensemble import VotingClassifier

evc1=VotingClassifier(estimators=[('cboost',catboost_model),('adb',adb_model),('bg',bg_model),('lgbm',lgbm_model)],voting='hard')
evc1.fit(X_train, y_train)
y_pred=evc.predict(X_test)
print(classification_report(y_test, y_pred))

evc1_metrics = metrics(X_test,evc1)
# Confusion maxtrix & metrics
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

# # # Result

In [ ]:
from prettytable import PrettyTable

Models =  ['KNN Classifier', 'Logostic Regression', 'Support Vector Machine','Naive Bayes','Decision Tree','Bagging Decision Tree','Boosting Decision Tree','Random Forest','Voting Classifier','CatBoost','LightGBM']
Test_scores = [96.49,98.24,95.61,97.36,94.73,95.61,96.49,96.49,97.36,98.24,98.24]
Train_scores = [92.30,95.60,96.92,93.62,100,99.12,100,100,98.02,100,100]
Chi_square_score=[97.36,96.49,97.36,95.61,95.61,97.36,96.49,97.36,'-',98.24,94.73]

Results = PrettyTable()

Results.add_column('Model', Models)
Results.add_column('Test_Score', Test_scores)
Results.add_column('Train_Score', Train_scores)
Results.add_column('Chi-square_Score', Chi_square_score)


print('Results: ')
print(Results)
